In [ ]:
import numpy as np

# Definición de la clase GraficayRed para manejar grafos y redes
class GraficayRed:
    def __init__(self, nodos, enlaces):
       """Definicion del constructor"""
       self.nodos = nodos  # Es la lista de nodos(paginas web)
       self.enlaces = enlaces #Es la lista de enlaces como tuplas(es origen,
                                                                    # destino)
       self.matriz = self._crear_matriz_adyacencia() # Crear matriz de
                                                           #adyacencia

    def _crear_matriz_adyacencia(self):
        """
        Se crea la matriz de adyacencia a partir de los nodos y enlaces.
        La matriz indicara la conexión entre los nodos, donde matriz[i][j] es
         el número de enlaces desde el nodo i al nodo j.

        Returns:
            np.ndarray: Matriz de adyacencia (n x n), donde n es el número de
            nodos.
        """
        n = len(self.nodos)  # Nos da el número de nodos
        matriz = np.zeros((n, n))  # Inicializa la matriz de ceros
        for (origen, destino) in self.enlaces:
            matriz[origen - 1][destino - 1] += 1  # Incrementa por cada enlace
                                                   #(origen -> destino)
        return matriz

    def obtener_matriz_probabilidad(self):
        """
        Lo que hara sera convertir la matriz de adyacencia en una matriz de
         probabilidad.
        Cada columna se normaliza dividiendo por la suma de sus valores,
        de modo que representen probabilidades.

        Returns:
            np.ndarray: Matriz de probabilidad (n x n).
        """
        n = len(self.nodos)
        matriz = self.matriz.copy()# Nos dara la copia d la matriz de adyacencia
        for j in range(n):
            suma_columna = np.sum(matriz[:, j])  # Suma los valores de la
                                                   #columna j
            if suma_columna > 0:
                matriz[:, j] /= suma_columna  # Normaliza la columna
                                                # dividiendo por la suma
        return matriz




In [ ]:
# Clase que implementa el algoritmo PageRank
class PageRank:
    def __init__(self, red, d=0.85):
        """
        Definicion del constructor de la clase PageRank.
        Inicializa el cálculo con la red dada y el factor de amortiguamiento d.

        Args:
            red (GraficayRed): Objeto que representa la red.
            d (float): Factor de amortiguamiento (por defecto, 0.85).
        """
        self.red = red  # Red (gráfico) proporcionado
        self.d = d  # Factor de amortiguamiento
        self.n = len(red.nodos)  # Número de nodos en la red
        self.matriz_proba = red.obtener_matriz_probabilidad()  # Matriz de
                                                                 #probabilidad
        self.matriz_transicion = self._crear_matriz_transicion()  # Matriz de
                                                                    #transición

    def _crear_matriz_de_transicion(self):
        """
        Crea la matriz de transición utilizada en el cálculo del PageRank.
        Aplica el factor de amortiguamiento d para combinar la matriz de
        probabilidad y la matriz uniforme.

        Returns:
            np.ndarray: Matriz de transición ajustada (n x n).
        """
        U = np.ones((self.n, self.n)) / self.n  # Matriz uniforme donde todos
                                                        # los valores son 1/n
        return self.d * self.matriz_proba + (1 - self.d) * U  # Combina probabi
                                                    #-lidad y amortiguamiento

    def calcular_pagerank(self, iteraciones=100, tol=1e-6, vector_inicial=None):
        """
        Calcula el PageRank utilizando un método iterativo.

        Args:
            iteraciones (int): Número máximo de iteraciones (por defecto, 100).
            tol (float): Tolerancia para determinar la convergencia
             (por defecto, 1e-6).
            vector_inicial (np.ndarray): Vector inicial para el cálculo
            (por defecto, uniforme).

        Returns:
            np.ndarray: Vector de PageRank con los valores finales para cada
             nodo.
        """
        if vector_inicial is None:
            vector_inicial = np.ones(self.n) / self.n  # Vector inicial uniforme
        vector_actual = vector_inicial  # Inicializa el vector actual
        for _ in range(iteraciones):
            nuevo_vector = np.dot(vector_actual, self.matriz_transicion)
             # Multiplicación vector-matriz
            # Verifica la convergencia comparando el cambio en norma 1
            if np.linalg.norm(nuevo_vector - vector_actual, 1) < tol:
                break  # Detiene si la diferencia es menor a la tolerancia
            vector_actual = nuevo_vector  # Actualiza el vector para la
                                               #próxima iteración
        return vector_actual


**PREGUNTAS**

(I) ¿Cual es la pagina web mas importante segun el algoritmo si se inicia con el vector (1/n,...,1/n)?

In [ ]:
import pandas as pd
import numpy as np
from google.colab import  drive

#  Leer los datos del archivo Excel
# Cargamos los datos del archivo proporcionado utilizando pandas
archivo_excel = ("/content/mnt/MyDrive/Web.xlsx")  # Ruta al archivo Excel
                                                     #proporcionado
data = pd.read_csv(archivo_excel, skiprows=0, sep="\t", names=["Website", "Index", "Cited by"])

#  Procesar los datos para extraer los nodos y enlaces
# Creamos la lista de nodos (índices de las páginas web)
nodos = data["Index"].tolist()

# Crear los enlaces desde los datos de la columna "Cited by"
enlaces = []
for _, row in data.iterrows():
    origen = row["Index"]  # Índice de la página origen
    destinos = str(row["Cited by"]).split(",")  # Dividimos las páginas citadas
    for destino in destinos:
        if destino.isdigit():  # Aseguramos que el destino es un índice válido
            enlaces.append((int(origen), int(destino.strip())))

#  Crear la red (grafo) utilizando la clase GraficayRed
red = GraficayRed(nodos, enlaces)

#  Inicializar el cálculo de PageRank
# Usamos el valor por defecto de d=0.85 y el vector inicial uniforme
pagerank = PageRank(red)

# Calcular el PageRank
resultado = pagerank.calcular_pagerank()

#  Identificar la página más importante
pagina_importante = nodos[np.argmax(resultado)]  # Índice de la página con
                                                   #el mayor PageRank
valor_pagerank = resultado[np.argmax(resultado)]  # Valor de PageRank más alto

#  Mostrar el resultado
print(f"La página más importante es la número {pagina_importante}, con un PageRank de {valor_pagerank:.4f}.")





(ii) Defina ahora un vector n de la siguiente forma:

In [ ]:
#  Crear un vector inicial que priorice las páginas ".ru"
# Iniciamos con un vector de ceros
vector_inicial = np.zeros(len(nodos))

# Lista de páginas con dominio ".ru" (de acuerdo con el archivo Excel)
pags_ru = [16, 17, 18, 19, 20, 21, 22, 23, 24, 25]  # Índices de las páginas ".ru"
s = len(pags_ru)  # Número total de páginas ".ru"

# Asignamos peso uniforme a las páginas ".ru"
for pag in pags_ru:
    vector_inicial[pag - 1] = 1 / s  # Distribuimos entre todas las páginas ".ru"

#  Calcular el PageRank usando este vector inicial
resultado_ru = pagerank.calcular_pagerank(vector_inicial=vector_inicial)

#  Encontrar la página más importante
pagina_importante_ru = nodos[np.argmax(resultado_ru)]

# Imprimimos el resultado
print(f"La página más importante con prioridad a '.ru' es la número {pagina_importante_ru}, con un PageRank de {resultado_ru[np.argmax(resultado_ru)]:.4f}.")



(iii)

In [ ]:
# Valores de d a probar
valores_d = [0.5, 0.85, 1.0]

# Iteramos sobre cada valor de d
for d in valores_d:
    # Inicializamos el PageRank con el valor actual de d
    pagerank_d = PageRank(red, d=d)

    # Calculamos el PageRank con vector inicial uniforme
    resultado_d = pagerank_d.calcular_pagerank()

    # Encontramos la página más importante para este valor de d
    pagina_importante_d = nodos[np.argmax(resultado_d)]

    # Imprimimos el resultado para este valor de d
    print(f"Con d={d}, la página más importante es la número {pagina_importante_d}, con un PageRank de {resultado_d[np.argmax(resultado_d)]:.4f}.")
